In [117]:
from bs4 import BeautifulSoup
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from time import sleep
from tqdm import tqdm
import glob
import unidecode
import pandas_gbq
from google.oauth2 import service_account
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import requests
import json

In [20]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    ##options.add_argument('--disable-dev-shm-usage')
    ##options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    ##options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    ##options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("prefs", {"download.default_directory": r"/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central"})

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)

    #driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [110]:
def get_centros(link_a_seccion):
    driver = driversetup()
    driver.get(link_a_seccion)
    sleep(2)
    html = driver.page_source # get the page source
    soup = BeautifulSoup(html, "html.parser")
    valores = soup.find_all('option', {'ng-value': 'sec.num'})
    centros = []
    for linea in valores:
        centros.append(int(linea.text))
    return centros

In [111]:
centros = get_centros("https://trep.gt/#!/tc4/DIV/e0/m1")
centros

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152]

In [112]:
def get_mesas(numero_centro):
    driver = driversetup()
    driver.get(f"https://trep.gt/#!/tc4/SEC/e0/m1?secNum={numero_centro}")
    sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    valores = soup.find_all('option', {'ng-value': 'cas.tipoId'})

    mesas_finales = []
    for linea in valores:
        mesas_finales.append(int(linea.text))

    return mesas_finales


In [115]:
datos_completos = []
for centro in centros:
    mesas = get_mesas(centro)
    datos_completos.append([centro, mesas])

In [119]:
datos_completos

[[1, [1, 2, 3, 4]],
 [2, [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],
 [3, [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]],
 [4, [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]],
 [5, [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]],
 [6, [64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]],
 [7, [82, 83, 84, 85, 86, 87]],
 [8,
  [88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111]],
 [9,
  [112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127]],
 [10,
  [128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   136,
   137,
   138,
   139,
   140,
   141,
   142,
   143,
   144,
   145,
   146,
   147,
   148]],
 [11, [149, 150, 151, 152, 153]],
 [12,
  [154,
   155,
   156,
   157,
   158,
   159,
   160,
   161,
   

In [118]:
with open("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/intermediate_data/centros_mesas.json", 'w') as f:
    json.dump(datos_completos, f, indent=2)


In [120]:
driver = driversetup()
driver.get(f"https://trep.gt/#!/tc4/CAS/e0/m1/2?casId=M5")
sleep(2)
images = driver.find_elements_by_tag_name('img')
images

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_tag_name'